# Исследование надежности заемщиков.

### Шаг 1. Обзор данных
Заказчик — кредитный отдел банка.

**Цель исследавания** - ответить на вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**
Входные данные от банка — статистика о платёжеспособности клиентов. О качестве данных ничего не известно. Поэтому перед началом исследования понадобится обзор данных.

Необходимо проверить данные на ошибки и оценить их влияние на исследование. Затем, на этапе предобработки найти возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

1. Обзор данных.
2. Предобработка данных.
3. Ответы на вопросы.

Составим первое представление о данных заказчика.

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получили первые 10 строк в таблице.

Теперь получим общую информацию о таблице:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. Тип данных в столбцах: `object`, `int`, `float`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита;

В названиях колонок нарушение стиля отсутствует.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о клиенте: пол, возраст, семейное положение,ежемесячных доход, цель получения кредита и тд. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Шаг 2.1 Заполнение пропусков

Посчитаем сколько в таблице пропущенных значений:

In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенны значение в столбцах `days_employed` и `total_income`. Доля пропущенных значений в каждом из столбцов с пропусками составлет 9.9%

Возможные причины появления пропусков - человеческий фактор. Некоторые клиенты могли не внести данные о ежемесячном доходе.

Пропуски в `total_income` могут помешать ходу исследования. Их необходимо заменить на медианное значение.Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства, поэтому идеальным для нашего варианта будет медианное.

In [4]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
df.isna().sum() #проверим что мы заполнили все пропуски в total_income

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Из таблицы видно и другие некорректные данные - аномалии. 

В нашем случае в столбце `days_employed` отрицательное колличество дней, данные очень большие для дней, люди не могут иметь стаж 100 лет. Логично предположить, что в таблице данные в часах. Нам следует перевести часы в сутки и избавится от минуса.

Такое могло произойти из-за человеческого фактора или технологичнских ошибок в ходе какого-то преобразования. Данный столбец не понадобится нам для исследования, но чтобы в будующем не сталкиваться с подобныи проблемами стоит предупредить коллег.

In [5]:
df['days_employed'] = abs(df['days_employed']) / 24
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,38.591076,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,119.966752,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,6.365815,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,288.744387,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,91.198185,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь заполним пропуски в `days_employed` медианным значением этого столбца и убедимся, что в таблице не осталось пропусков. Для этого еще раз посчитаем пропущенные значения.

In [6]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

В столбце `children` обнаружено значение -1 и 20. Избавимся от минуса, а вместо аномально большого значения 20 детей укажем 2 т.к возможно "0" был напечатан случайно.

In [7]:
df['children'] = abs(df['children'])
df.loc[df['children'] == 20, 'children'] = 2
df['children'].value_counts() #проверим что ушли отрицательные значения и выброс

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце `total_income` на целочисленный:

In [8]:
df['total_income'] = df['total_income'].astype('int')
df.info() #проверим изменение типа

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Проверим данные на наличие явных дубликатов. 

In [9]:
df.duplicated().sum()

54

Чтобы избавиться от таких дубликатов, вызовем метод drop_duplicates():

In [10]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum() #проверим, что удалили явные дубликаты

0

Теперь проверим наличие неявных дубликатов в стобце `education`

In [11]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Одни и те же значения, но записанны по-разному: с использованием заглавных и строчных букв. Приведем их к одному регистру.

In [12]:
df['education'] = df['education'].str.lower()
df['education'].value_counts() # проверим, что удалили неявные дубликаты

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Для поиска явных дубликатов в pandas есть специальный метод duplicated(). Чтобы посчитать такие строки, результат метода передают функции sum(). Чтобы избавиться от таких дубликатов, вызовем метод drop_duplicates()

Для поиска неявных дубликатов вызвали метод value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке. Для их удаления перевели все к нижнему регистру методом str.lower()

Появление таких дубликатов возможно из-за человеческого фактара или неккоректной выгрузки

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
* каждому уникальному значению из `education` соответствует уникальное значение `education_id` — в первом;
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором

In [13]:
df_education = df[['education', 'education_id']]
df_education.head(10)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
5,высшее,0
6,высшее,0
7,среднее,1
8,высшее,0
9,среднее,1


In [14]:
df_family_status = df[['family_status', 'family_status_id']]
df_family_status.value_counts()

family_status          family_status_id
женат / замужем        0                   12344
гражданский брак       1                    4163
Не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
dtype: int64

Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. 

In [15]:
df.drop(columns=['education','family_status'], axis=1, inplace=True)
df.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### Шаг 2.6. Категоризация дохода.

На основании диапазонов, создадим столбец `total_income_category`:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Cоздадим функцию, которая будет присваивать категории:

In [16]:
def income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if (total_income >= 30001)&(total_income <= 50000):
        return 'D'
    if (total_income >= 50001)&(total_income <= 200000):
        return 'C'
    if (total_income >= 200001)&(total_income <= 1000000):
        return 'B'
    return 'A'
income_category(70000)  

'C'

Написанная функция работает корректно. Осталось создать отдельный столбец с категориями дохода, и в его ячейках записать значения, возвращаемые функцией. 

In [17]:
df['total_income_category'] = df['total_income'].apply(income_category)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Категоризация поможет нам изучить влияние уровня дохода на возврат кредита в срок.

### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.


In [18]:
def purpose_group(purpose): #свадьб недвижимост жиль автомобил образовани
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if ('недвижимост' in purpose) or ('жиль' in purpose):
        return 'операции с недвижимостью'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    return 'получение образования'
purpose_group('сыграть свадьбу') #проверим что функция работает корректно

'проведение свадьбы'

Создадим новый столбец `purpose_category`:

In [19]:
df['purpose_category'] = df['purpose'].apply(purpose_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Категоризация поможет нам изучить влияние целей кредита на возврат кредита в срок

### Ответы на вопросы.

##### Вопрос 1:
**Есть ли зависимость между количеством детей и возвратом кредита в срок?**


Напишем функцию для категоризации клиентов с детьми и без детей.

In [20]:
def having_children(children):
    if children > 0:
        return 1
    return 0
having_children(3) #проверим работу функции

1

Создадим новый столец `having_child`

In [21]:
df['having_child'] = df['children'].apply(having_children)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,having_child
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,0
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,1
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,0


Создадим сводную таблицу, чтобы посмотреть зависимость между наличием детей и возврата кредита в срок.

In [22]:
df_pivot_child = df.pivot_table(index=['having_child'],values='debt')
df_pivot_child

,debt
having_child,
0,0.075353
1,0.092070


* Вероятность задолженности клиента с детьми составляет 0.092070 или **9.2%**
* Вероятность задолженности клиента без детей составляет 0.075353 или **7.5%**

##### Вывод 1:

Исходя из данных выше можно сделать вывод, что клиенты без детей ответственнее относятся к выплатам кредита в срок. 

##### Вопрос 2:
**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Создадим сводную таблицу и посмотрим зависимость между семейным положением и возвратом кредита в срок. Категоризация по семейному положению в столбце `family_status_id`

* 0 - женат/замужем 
* 1 - гражданский брак
* 2 - вдовец/вдова 
* 3 - в разводе
* 4 - Не женат/не замужем

In [23]:
df_pivot_family = df.pivot_table(index=['family_status_id'], values='debt')
df_pivot_family

,debt
family_status_id,
0,0.075421
1,0.093202
2,0.065693
3,0.071130
4,0.097509


Вероятность задолженности по убыванию:

1. не женат / не замужем = 9,8%
2. гражданский брак = 9,3%
3. женат / замужем = 7,5%
4. в разводе = 7,1%
5. вдовец / вдова = 6,6%

##### Вывод 2:
На основании этого можно сделать вывод, что зависимость между семейным положением и возвратом кредита в срок есть. Люди не в браке и не состощие в официальном браке, имеют больший шанс задолжить выплаты по кредиту. Также из таблицы видно, что те, кто развелись и овдовели чаще платят в срок.

##### Вопрос 3:
**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Создадим сводную таблицу и посмотрим зависимость между уровнем дохода и возвратом кредита в срок. В этом нам поможет столбец `total_income_category` с категоризацией дохода
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [24]:
df_pivot_income = df.pivot_table(index=['total_income_category'], values='debt')
df_pivot_income

,debt
total_income_category,
A,0.080000
B,0.070621
C,0.084825
D,0.060000
E,0.090909


Получившиеся вероятности:
1. Категория Е - 9,1%
2. Категория С - 8,4%
3. Категория А - 8%
4. Категория В - 7,1%
5. Категория D - 6%


##### Вывод 3:
Зависимость между уровнем дохода и возвратом кредита в срок не наблюдается. 

##### Вопрос 4:
**Как разные цели кредита влияют на его возврат в срок?**

Создадим сводную таблицу. В ответе на этот вопрос нам поможет столбец `purpose_category`

In [25]:
df_pivot_purpose = df.pivot_table(index=['purpose_category'], values='debt')
df_pivot_purpose                                 

,debt
purpose_category,
операции с автомобилем,0.093547
операции с недвижимостью,0.072314
получение образования,0.092177
проведение свадьбы,0.079657


Полученные вероятности:
1. операции с автомобилем 9,4%
2. получение образования 9,2%
3. проведение свадьбы 8%
4. операции с недвижимостью 7,2%

##### Вывод 4:
На основании полученных данных можно сделать вывод, что клиенты берущие кредит на операции с недживимостью чаще платят в срок. Чуть реже на проведение свадьбы. Клиенты берущие кредит для операций с автомобилем и получения образования чаще задерживают выплаты.

## Общий вывод:

Семейный статус влияет на возврат кредита в срок: люди, состоящие в официальном браке, разведенные или потерявшие своего супруга реже пропускают выплаты. 
Наличие детей также влияет на выплаты в срок: так люди не имеющие детей имеют меньше задоленности по кредиту.
Цели кредита также влияют на возврат в срок: люди, берущие кредит на операции с недвижимостью или на проведение свадьбы имеют меньше задолженностей.

На основании этого можно составить портрет идеального клиента:
Женщина( или мужчина, влияние пола не проводилось в данном иследовании) замужем или в разводе, без детей, берет кредит с целью покупки недвижимости.